# Creating a basic chat experience with kernel arguments

In this example, we show how you can build a simple chat bot by sending and updating the kernel arguments with your requests.

We introduce the Kernel Arguments object which in this demo functions similarly as a key-value store that you can use when running the kernel.

The chat history is local (i.e. in your computer's RAM) and not persisted anywhere beyond the life of this Jupyter session.

In future examples, we will show how to persist the chat history on disk so that you can bring it into your applications.

In this chat scenario, as the user talks back and forth with the bot, the chat context gets populated with the history of the conversation. During each new run of the kernel, the kernel arguments and chat history can provide the AI with its variables' content.


Initial configuration for the notebook to run properly.

In [2]:
# Make sure paths are correct for the imports

import os
import sys

notebook_dir = os.path.abspath("")
parent_dir = os.path.dirname(notebook_dir)
grandparent_dir = os.path.dirname(parent_dir)


sys.path.append(grandparent_dir)

We will load our settings and get the LLM service to use for the notebook.

In [3]:
from services import Service

from service_settings import ServiceSettings

service_settings = ServiceSettings.create()

# Select a service to use for this notebook (available services: OpenAI, AzureOpenAI, HuggingFace)
selectedService = (
    Service.AzureOpenAI
    if service_settings.global_llm_service is None
    else Service(service_settings.global_llm_service.lower())
)
print(f"Using service type: {selectedService}")

Using service type: Service.AzureOpenAI


In [4]:
from semantic_kernel import Kernel

kernel = Kernel()

service_id = None
if selectedService == Service.OpenAI:
    from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

    service_id = "default"
    kernel.add_service(
        OpenAIChatCompletion(
            service_id=service_id,
            env_file_path="../.env",
        ),
    )
elif selectedService == Service.AzureOpenAI:
    from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

    service_id = "default"
    kernel.add_service(
        AzureChatCompletion(
            service_id=service_id,
            env_file_path="../.env",
        ),
    )

Let's define a prompt outlining a dialogue chat bot.


In [5]:
prompt = """
챗봇은 모든 주제에 대해 사용자와 대화를 나눌 수 있습니다.
챗봇은 명확한 지침을 제공하거나 답변이 없는 경우 '모르겠습니다'라고 말할 수 있습니다.

{{$history}}
User: {{$user_input}}
ChatBot: """

Register your semantic function


In [ ]:
from semantic_kernel.connectors.ai.open_ai import AzureChatPromptExecutionSettings, OpenAIChatPromptExecutionSettings
from semantic_kernel.prompt_template import PromptTemplateConfig
from semantic_kernel.prompt_template.input_variable import InputVariable

if selectedService == Service.OpenAI:
    execution_settings = OpenAIChatPromptExecutionSettings(
        service_id=service_id,
        ai_model_id="gpt-3.5-turbo",
        max_tokens=2000,
        temperature=0.7,
    )
elif selectedService == Service.AzureOpenAI:
    execution_settings = AzureChatPromptExecutionSettings(
        service_id=service_id,
        ai_model_id="gpt-35-turbo",
        max_tokens=2000,
        temperature=0.7,
    )

prompt_template_config = PromptTemplateConfig(
    template=prompt,
    name="chat",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="user_input", description="The user input", is_required=True),
        InputVariable(name="history", description="The conversation history", is_required=True),
    ],
    execution_settings=execution_settings,
)

chat_function = kernel.add_function(
    function_name="chat",
    
    plugin_name="chatPlugin",
    prompt_template_config=prompt_template_config,
)

In [7]:
from semantic_kernel.contents import ChatHistory

chat_history = ChatHistory()
chat_history.add_system_message("당신은 책을 추천하는 데 능숙한 유용한 챗봇입니다.")

Initialize the Kernel Arguments


In [8]:
from semantic_kernel.functions import KernelArguments

arguments = KernelArguments(user_input="안녕하세요, 추천 도서를 찾고 있습니다.", history=chat_history)

Chat with the Bot


In [9]:
response = await kernel.invoke(chat_function, arguments)
print(response)

안녕하세요! 어떤 종류의 책을 찾고 계신가요? 소설, 비소설, 자기계발, 역사, 과학 등 어떤 장르를 선호하시는지 알려주시면 추천해 드리겠습니다.


Update the history with the output


In [10]:
chat_history.add_assistant_message(str(response))

Keep Chatting!


In [11]:
async def chat(input_text: str) -> None:
    # Save new message in the context variables
    print(f"User: {input_text}")

    # Process the user message and get an answer
    answer = await kernel.invoke(chat_function, KernelArguments(user_input=input_text, history=chat_history))

    # Show the response
    print(f"ChatBot: {answer}")

    chat_history.add_user_message(input_text)
    chat_history.add_assistant_message(str(answer))

In [12]:
await chat("저는 역사와 철학을 좋아하고 그리스에 대해 새로운 것을 배우고 싶은데, 어떤 것을 추천해 주시겠어요?")

User: 저는 역사와 철학을 좋아하고 그리스에 대해 새로운 것을 배우고 싶은데, 어떤 것을 추천해 주시겠어요?
ChatBot: 그리스 역사와 철학에 대해 관심이 있으시다면 다음 책들을 추천드립니다:

1. **"The Histories" by Herodotus** - 그리스의 아버지로 불리는 헤로도토스의 저서로, 고대 그리스의 역사와 문화를 이해하는 데 아주 좋은 자료입니다.

2. **"The Peloponnesian War" by Thucydides** - 펠로폰네소스 전쟁에 대한 투키디데스의 기록으로, 그리스 역사와 정치에 대한 깊은 이해를 제공해줍니다.

3. **"The Republic" by Plato** - 플라톤의 대표작으로, 철학적인 대화 형식으로 정의와 이상 국가에 대해 논의합니다.

4. **"Nicomachean Ethics" by Aristotle** - 아리스토텔레스의 윤리학 작품으로, 인간의 행복과 덕에 대해 깊이 있게 탐구합니다.

5. **"The Birth of Tragedy" by Friedrich Nietzsche** - 니체가 그리스 비극을 통해 예술과 철학에 대해 논의한 책입니다.

이 책들은 그리스의 역사와 철학을 깊이 있게 이해하는 데 많은 도움을 줄 것입니다. 즐거운 독서 되세요!


In [13]:
await chat("흥미롭게 들리는데 무슨 내용인가요?")

User: 흥미롭게 들리는데 무슨 내용인가요?
ChatBot: 물론입니다! 각 책의 내용을 간단히 설명해드리겠습니다.

1. **"The Histories" by Herodotus**:
   이 책은 고대 그리스의 역사가 헤로도토스가 쓴 책으로, 그리스-페르시아 전쟁을 중심으로 그 시대의 역사와 문화를 다룹니다. 헤로도토스는 다양한 지역과 민족의 이야기를 수집하여 기록했으며, 그리스와 페르시아 간의 충돌을 상세하게 묘사합니다. 이 책은 역사서의 기초를 닦았다고 평가받습니다.

2. **"The Peloponnesian War" by Thucydides**:
   투키디데스는 펠로폰네소스 전쟁(기원전 431-404년)을 기록한 그리스 역사가입니다. 이 책은 아테네와 스파르타 간의 전쟁을 다루며, 전쟁의 원인, 과정, 결과를 분석합니다. 투키디데스는 객관적이고 사실적인 서술로 유명하며, 그의 기록은 정치와 전쟁의 본질을 이해하는 데 중요한 자료로 평가받습니다.

3. **"The Republic" by Plato**:
   플라톤의 "국가"는 철학적 대화 형식으로 쓰인 책으로, 정의, 이상 국가, 개인과 사회의 관계 등에 대해 논의합니다. 소크라테스를 주인공으로 하여 다양한 인물들과의 대화를 통해 철학적 개념을 탐구하며, 이상적인 국가 구조와 철학자의 역할에 대해 설명합니다.

4. **"Nicomachean Ethics" by Aristotle**:
   아리스토텔레스의 "니코마코스 윤리학"은 인간의 행복과 덕에 대해 다루는 철학서입니다. 아리스토텔레스는 행복을 최고선으로 정의하고, 이를 달성하기 위한 덕(아레테)의 중요성을 강조합니다. 이 책은 윤리학의 중요한 기초를 제공하며, 인간의 삶과 행동에 대한 깊은 통찰을 제공합니다.

5. **"The Birth of Tragedy" by Friedrich Nietzsche**:
   니체의 "비극의 탄생"은 그리스 비극을 통해 예술과 철학에 대한 논의를 담고 있습니다. 니체는 아폴론적 요소(이성, 질서)와 디오니소

In [14]:
await chat("그 책을 읽으면 그리스 역사에 대해 정확히 무엇을 배울 수 있을까요?")

User: 그 책을 읽으면 그리스 역사에 대해 정확히 무엇을 배울 수 있을까요?
ChatBot: 이 책들을 읽으면 그리스 역사에 대해 다양한 측면을 배울 수 있습니다. 각 책이 제공하는 주요한 학습 내용은 다음과 같습니다:

1. **"The Histories" by Herodotus**:
   - 그리스-페르시아 전쟁의 원인, 과정, 결과
   - 고대 그리스와 페르시아 제국의 정치, 문화, 사회 구조
   - 다양한 지역과 민족의 풍습과 전통
   - 역사 기록의 중요성과 방법론

2. **"The Peloponnesian War" by Thucydides**:
   - 아테네와 스파르타 간의 펠로폰네소스 전쟁의 상세한 기록
   - 전쟁의 정치적, 경제적, 사회적 영향
   - 전쟁 중 발생한 주요 사건과 인물
   - 전쟁의 원인과 결과에 대한 분석

3. **"The Republic" by Plato**:
   - 이상적인 국가와 정부 구조에 대한 플라톤의 철학적 견해
   - 정의와 인간의 도덕적 삶에 대한 논의
   - 철학자가 국가를 이끌어야 한다는 철인 통치자의 개념
   - 교육, 예술, 사회 제도에 대한 플라톤의 사상

4. **"Nicomachean Ethics" by Aristotle**:
   - 인간의 행복(에우다이모니아)와 이를 달성하기 위한 덕(아레테)
   - 윤리적 덕과 지적 덕의 차이
   - 중용(중도)의 개념과 윤리적 삶의 중요성
   - 개인의 도덕적 발달과 사회적 책임

5. **"The Birth of Tragedy" by Friedrich Nietzsche**:
   - 그리스 비극의 본질과 예술적 가치
   - 아폴론적 요소와 디오니소스적 요소의 융합
   - 예술과 삶의 의미에 대한 철학적 통찰
   - 고대 그리스 문화와 철학이 현대에 미친 영향

이 책들을 통해 고대 그리스의 다양한 역사적 사건, 문화, 철학적 사상, 사회 구조 등을 깊이 있게 이해할 수 있을 것입니다. 각각의 저자는 그리스 역사와 철학의 

In [15]:
await chat("이 주제에 대해 읽을 만한 책을 몇 권 더 추천해 주시겠어요?")

User: 이 주제에 대해 읽을 만한 책을 몇 권 더 추천해 주시겠어요?
ChatBot: 물론입니다! 그리스 역사와 철학에 대해 더 깊이 탐구하고 싶으시다면 다음 책들도 추천드립니다:

1. **"The Iliad" and "The Odyssey" by Homer**:
   - 호메로스의 서사시로, 트로이 전쟁과 영웅 오디세우스의 모험을 다룹니다. 그리스 신화와 영웅 서사시의 중요한 자료입니다.

2. **"Lives of the Noble Greeks and Romans" by Plutarch**:
   - 플루타르코스의 저서로, 그리스와 로마의 위대한 인물들의 전기를 다룹니다. 역사적 인물들의 삶과 업적을 비교하면서 그들의 성격과 도덕적 품성을 탐구합니다.

3. **"Meditations" by Marcus Aurelius**:
   - 로마 황제이자 철학자인 마르쿠스 아우렐리우스의 개인적인 명상록으로, 스토아 철학의 중요한 작품입니다. 그리스 철학이 로마 시대에 어떻게 수용되고 발전했는지를 이해하는 데 도움이 됩니다.

4. **"The Symposium" by Plato**:
   - 플라톤의 또 다른 대화 형식의 저서로, 사랑과 미의 본질에 대해 논의합니다. 그리스 철학자들이 사랑과 인간 관계에 대해 어떻게 생각했는지를 알 수 있습니다.

5. **"Antigone" by Sophocles**:
   - 소포클레스의 비극으로, 개인의 도덕적 의무와 국가 법의 충돌을 다룹니다. 그리스 비극의 중요한 예시로서, 그리스 문학과 철학적 주제를 이해하는 데 도움이 됩니다.

6. **"The Politics" by Aristotle**:
   - 아리스토텔레스의 정치 철학 저서로, 다양한 정치 체제와 국가의 역할에 대해 논의합니다. 고대 그리스의 정치적 사상과 사회 구조를 깊이 있게 이해할 수 있습니다.

7. **"Greek Religion" by Walter Burkert**:
   - 고대 그리스 종교와 신화에 대한 학술적인 저서로, 그리스 신화와 종교

After chatting for a while, we have built a growing history, which we are attaching to each prompt and which contains the full conversation. Let's take a look!


In [16]:
print(chat_history)

<chat_history><message role="system"><text>당신은 책을 추천하는 데 능숙한 유용한 챗봇입니다.</text></message><message role="assistant"><text>안녕하세요! 어떤 종류의 책을 찾고 계신가요? 소설, 비소설, 자기계발, 역사, 과학 등 어떤 장르를 선호하시는지 알려주시면 추천해 드리겠습니다.</text></message><message role="user"><text>저는 역사와 철학을 좋아하고 그리스에 대해 새로운 것을 배우고 싶은데, 어떤 것을 추천해 주시겠어요?</text></message><message role="assistant"><text>그리스 역사와 철학에 대해 관심이 있으시다면 다음 책들을 추천드립니다:

1. **"The Histories" by Herodotus** - 그리스의 아버지로 불리는 헤로도토스의 저서로, 고대 그리스의 역사와 문화를 이해하는 데 아주 좋은 자료입니다.

2. **"The Peloponnesian War" by Thucydides** - 펠로폰네소스 전쟁에 대한 투키디데스의 기록으로, 그리스 역사와 정치에 대한 깊은 이해를 제공해줍니다.

3. **"The Republic" by Plato** - 플라톤의 대표작으로, 철학적인 대화 형식으로 정의와 이상 국가에 대해 논의합니다.

4. **"Nicomachean Ethics" by Aristotle** - 아리스토텔레스의 윤리학 작품으로, 인간의 행복과 덕에 대해 깊이 있게 탐구합니다.

5. **"The Birth of Tragedy" by Friedrich Nietzsche** - 니체가 그리스 비극을 통해 예술과 철학에 대해 논의한 책입니다.

이 책들은 그리스의 역사와 철학을 깊이 있게 이해하는 데 많은 도움을 줄 것입니다. 즐거운 독서 되세요!</text></message><message role="user"><text>흥미롭게 들리는데 무슨 내용인가요?</text></message><me